In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

# Premiers pas 

Pour l'affichage, on utilise `println` le plus souvent : 

In [ ]:
println("Vive Julia !")

Les types sont importants pour Julia : on ne peut pas tout concaténer comme en JavaScript.

In [ ]:
println("Vive Julia " * VERSION * " !")
# La constante VERSION a pour type VersionNumber.

D'où la conversion : 

In [ ]:
println("Vive Julia " * string(VERSION) * " !")

Quand on définit une variable, l'interpréteur trouve le type le plus précis : 

In [ ]:
a = 42
println(typeof(a))
    
b = 3.14159
println(typeof(b))
    
c = 'c'
println(typeof(c))
    
d = "Julia"
println(typeof(d))

Une variable peut changer de type au cours de sa vie :

In [ ]:
a = 42
println(typeof(a))
a = 'c'
println(typeof(a))

Pour le débogage, @show affiche aussi l'expression :

In [ ]:
@show a;

# Une bafouille sur les chaînes de caractères

La concaténation ne fait pas de conversion automatique, mais l'interpolation bien : 

In [ ]:
langage = "Julia"
println("Vive $langage $(VERSION) !")

Unicode est géré nativement comme si de rien n'était (Python 2, où te caches-tu ?) -- même pour les noms de variable :

In [ ]:
γs_regex = r"γ|Γ" # \gamma et \Gamma dans l'interpréteur, Jupyter et la plupart des environnements
match(γs_regex, "abγ")

# Structures de données de base

Les tableaux de base peuvent avoir un type pour les éléments (plus efficace, comme NumPy) ou autoriser tous les types (comme Python) ; si aucun type n'est spécifié, Julia utilise le type le plus précis : 

In [ ]:
a = Int[1, 2, 3]

In [ ]:
# Int[1, 2, 3.14159] : erreur !

In [ ]:
b = [1, "2", 3.14159] # Type inféré : Any

In [ ]:
c = [1, 2, 3] # Type inféré : Int

Les fonctions dont le nom se termine par `!` modifient leur premier argument (conventionnellement), toujours en respectant les types :

In [ ]:
push!(a, 4)

In [ ]:
push!(a, ℯ) # \euler

In [ ]:
push!(b, ℯ)

Pour des dictionnaires, on a `Dict{K,V}` :

In [ ]:
d = Dict{Int, Float64}(1 => π, 2 => ℯ)